In [1]:
!git clone 'https://github.com/aakashvardhan/s7-in-depth-practice.git'

Cloning into 's7-in-depth-practice'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 123 (delta 58), reused 104 (delta 39), pack-reused 0
Receiving objects: 100% (123/123), 332.38 KiB | 671.00 KiB/s, done.
Resolving deltas: 100% (58/58), done.


In [2]:
import sys
sys.path.append('/content/s7-in-depth-practice')
sys.path.append('/content/s7-in-depth-practice/models')

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from model_utils import model_summary, test_model_sanity, sgd_optimizer
from models.model import model3

from utils import train, test, get_device, plt_fig, get_incorrect_predictions, plot_incorrect

In [4]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      transforms.Resize((28, 28)),
                                      transforms.RandomAffine(degrees=20, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                                      transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

In [5]:
train_data = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

100%|██████████| 9912422/9912422 [00:00<00:00, 193605025.70it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 10706707.96it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 81260179.97it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 19741480.59it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
# CUDA?
cuda = get_device()

# For reproducibility
torch.manual_seed(1)

if torch.cuda.is_available():
    torch.cuda.manual_seed(1)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_data, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_data, **dataloader_args)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
model3 = model3().to(cuda)

In [8]:
model_summary(model3)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 26, 26]              36
              ReLU-2            [-1, 4, 26, 26]               0
       BatchNorm2d-3            [-1, 4, 26, 26]               8
           Dropout-4            [-1, 4, 26, 26]               0
         ConvBlock-5            [-1, 4, 26, 26]               0
            Conv2d-6            [-1, 8, 24, 24]             288
              ReLU-7            [-1, 8, 24, 24]               0
       BatchNorm2d-8            [-1, 8, 24, 24]              16
           Dropout-9            [-1, 8, 24, 24]               0
        ConvBlock-10            [-1, 8, 24, 24]               0
           Conv2d-11           [-1, 16, 22, 22]           1,152
             ReLU-12           [-1, 16, 22, 22]               0
      BatchNorm2d-13           [-1, 16, 22, 22]              32
          Dropout-14           [-1, 16,

In [9]:
from torch.optim.lr_scheduler import StepLR


optimizer = sgd_optimizer(model3, 0.04)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)
EPOCHS = 15
lr = []
for epoch in range(1,EPOCHS+1):
    print("EPOCH:", epoch)
    train(model3, cuda, train_loader, optimizer, epoch)
    test(model3, cuda, test_loader)
    scheduler.step()
    lr.append(optimizer.param_groups[0]['lr'])

EPOCH: 1


Loss=0.1172758936882019 Batch_id=468 Accuracy=84.58: 100%|██████████| 469/469 [00:39<00:00, 11.82it/s]



Test set: Average loss: 0.0695, Accuracy: 9794/10000 (97.94%)

EPOCH: 2


Loss=0.09736791998147964 Batch_id=468 Accuracy=95.79: 100%|██████████| 469/469 [00:40<00:00, 11.62it/s]



Test set: Average loss: 0.0519, Accuracy: 9837/10000 (98.37%)

EPOCH: 3


...


Test set: Average loss: 0.0202, Accuracy: 9937/10000 (99.37%)

EPOCH: 14


Loss=0.0671563670039177 Batch_id=468 Accuracy=98.33: 100%|██████████| 469/469 [00:40<00:00, 11.56it/s]



Test set: Average loss: 0.0201, Accuracy: 9936/10000 (99.36%)

EPOCH: 15


Loss=0.07074109464883804 Batch_id=468 Accuracy=98.38: 100%|██████████| 469/469 [00:39<00:00, 11.76it/s]



Test set: Average loss: 0.0200, Accuracy: 9938/10000 (99.38%)



In [10]:
# Save model
torch.save(model3.state_dict(), 'best_model.pth')